In [44]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math as math

from scipy.io import loadmat
import networkx as nx


import sys

import matplotlib.pyplot as plt
import networkx as nx
from bokeh.io import show, output_notebook
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, StaticLayoutProvider
from bokeh.models.graphs import from_networkx

output_notebook()
%matplotlib inline

Loading BokehJS ...

In [45]:
connectivity = loadmat("../data/aux_meanACS69.mat")['meanACS']
region_names = pd.read_csv('../data/dk_names.csv')['Atlas']
coords = pd.read_csv('../data/dk.csv').set_index('Name')

conn = pd.DataFrame(connectivity)
conn.columns = list(region_names)
conn.index = list(region_names)

In [46]:
for region in region_names:
    if region not in coords.index:
        conn = conn.drop(index=region, columns=region)

# clip out any connectivity below 2
cutoff = 2.
conn = conn.clip(lower=cutoff).replace(cutoff, 0)

coords = coords.sort_values(by=['hemisphere','lobe'])

In [47]:
radius = 1.0
positions = {}
shift = 2*np.pi/136
for i in range(68):
    region = coords.index[i]
    if i < 34:
        x = radius*np.cos(2*np.pi*i/68 + shift)
        y = radius*np.sin(2*np.pi*i/68 + shift)
    else:
        i = i - 33
        x = radius*np.cos(-2*np.pi*i/68 + shift)
        y = radius*np.sin(-2*np.pi*i/68 + shift)
    positions.update({region:[x,y]})

In [48]:
G = nx.from_pandas_adjacency(conn)

In [130]:
import networkx as nx

from bokeh.io import show, output_file
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4

plot = Plot(plot_width=900, plot_height=500,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))
plot.title.text = "Brain Connectivity - DK Atlas"

plot.add_tools(HoverTool(tooltips=[("", "@name"),("", "@nl")]), TapTool(), BoxSelectTool())

graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0,0))

graph_renderer.node_renderer.glyph = Circle(size=12, fill_color=Spectral4[0])
graph_renderer.node_renderer.selection_glyph = Circle(size=15, fill_color=Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size=15, fill_color=Spectral4[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width=3)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=3)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=3)

graph_renderer.selection_policy = NodesAndLinkedEdges()
# graph_renderer.inspection_policy = EdgesAndLinkedNodes()

fixed_layout_provider = StaticLayoutProvider(graph_layout=positions)
graph_renderer.layout_provider = fixed_layout_provider
graph_renderer.node_renderer.data_source.data.update({'name':graph_renderer.node_renderer.data_source.data['index']})
graph_renderer.node_renderer.data_source.data.update({'nl':neighbor_list})

plot.renderers.append(graph_renderer)

output_notebook()
show(plot)

# output_file("interactive_graphs.html")

Loading BokehJS ...

In [117]:
neighbor_list = []
for region in graph_renderer.node_renderer.data_source.data['index']:
    neighbor_list.append([n for n in G.neighbors(region)])

In [118]:
neighbor_list

[['ctx-lh-inferiorparietal',
  'ctx-lh-inferiortemporal',
  'ctx-lh-middletemporal',
  'ctx-lh-superiortemporal',
  'ctx-lh-supramarginal'],
 ['ctx-lh-posteriorcingulate',
  'ctx-lh-rostralanteriorcingulate',
  'ctx-lh-superiorfrontal',
  'ctx-rh-caudalanteriorcingulate'],
 ['ctx-lh-parsopercularis',
  'ctx-lh-precentral',
  'ctx-lh-rostralmiddlefrontal',
  'ctx-lh-superiorfrontal'],
 ['ctx-lh-lateraloccipital',
  'ctx-lh-lingual',
  'ctx-lh-pericalcarine',
  'ctx-lh-precuneus',
  'ctx-lh-superiorparietal',
  'ctx-rh-cuneus'],
 ['ctx-lh-fusiform',
  'ctx-lh-inferiortemporal',
  'ctx-lh-parahippocampal',
  'ctx-lh-temporalpole'],
 ['ctx-lh-entorhinal',
  'ctx-lh-inferiorparietal',
  'ctx-lh-inferiortemporal',
  'ctx-lh-lateraloccipital',
  'ctx-lh-lingual',
  'ctx-lh-middletemporal',
  'ctx-lh-parahippocampal',
  'ctx-lh-superiortemporal',
  'ctx-lh-temporalpole',
  'ctx-lh-insula'],
 ['ctx-lh-bankssts',
  'ctx-lh-fusiform',
  'ctx-lh-inferiortemporal',
  'ctx-lh-lateraloccipital',
  'c

### todo
- add [Bezier](https://bokeh.pydata.org/en/latest/docs/user_guide/graph.html)